![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Plan your trip with Kayak 

## Company's description 📇

<a href="https://www.kayak.com" target="_blank">Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href="https://www.bookingholdings.com/" target="_blank">Booking Holdings</a> which now holds: 

* <a href="https://booking.com/" target="_blank">Booking.com</a>
* <a href="https://kayak.com/" target="_blank">Kayak</a>
* <a href="https://www.priceline.com/" target="_blank">Priceline</a>
* <a href="https://www.agoda.com/" target="_blank">Agoda</a>
* <a href="https://Rentalcars.com/" target="_blank">RentalCars</a>
* <a href="https://www.opentable.com/" target="_blank">OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">One Week In.com</a> here are the top-35 cities to visit in France: 

```python 
["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]
```

Your team should focus **only on the above cities for your project**. 


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

## Deliverable 📬

To complete this project, your team should deliver:

* A `.csv` file in an S3 bucket containing enriched information about weather and hotels for each french city

* A SQL Database where we should be able to get the same cleaned data from S3 

* Two maps where you should have a Top-5 destinations and a Top-20 hotels in the area. You can use plotly or any other library to do so. It should look something like this: 

![Map](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/Kayak_best_destination_project.png)

In [467]:
import pandas as pd
import requests
import pprint


In [468]:
# Create list of the 35 cities
cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [469]:
# Store cities in a dataframe
df = pd.DataFrame(cities, columns=['City'])
df

,City
0,Mont Saint Michel
1,St Malo
2,Bayeux
3,Le Havre
4,Rouen
5,Paris
6,Amiens
7,Lille
8,Strasbourg
9,Chateau du Haut Koenigsbourg


### Get weather data from each destination
##### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### I : Get GPS coordinates of 35 TOP cities from nominatim.org API

In [470]:
# Get info for one city, here Ariege
get_one_city = requests.get('https://nominatim.openstreetmap.org/search?q=Ariege&country=France&format=json&limit=1')
get_one_city.json()

[{'place_id': 281653603,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 7439,
  'boundingbox': ['42.5732416', '43.3162514', '0.8267506', '2.1758135'],
  'lat': '42.9455368',
  'lon': '1.4065544156065486',
  'display_name': 'Ariège, Occitanie, France métropolitaine, France',
  'class': 'boundary',
  'type': 'administrative',
  'importance': 0.6009114788084189,
  'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_boundary_administrative.p.20.png'}]

In [471]:
# get Lat and Lon for Ariege
print(get_one_city.json()[0]['lat'])
print(get_one_city.json()[0]['lon'])

42.9455368
1.4065544156065486


In [472]:
list_lat = []
list_lon = []
for city in cities:
    city_encoded = city.replace(' ','+')
    info_city = requests.get('https://nominatim.openstreetmap.org/search?q={}&country=France&format=json&limit=1'.format(city_encoded))
    gps_data = info_city.json()
    if gps_data ==[]:
        list_lat.append('Null')
        list_lon.append('Null')
        print("Info for {} 'null' ".format(city))
    else:
        list_lat.append(gps_data[0]['lat'])
        list_lon.append(gps_data[0]['lon'])
        print("Info for {} ok".format(city))

Info for Mont Saint Michel ok
Info for St Malo ok
Info for Bayeux ok
Info for Le Havre ok
Info for Rouen ok
Info for Paris ok
Info for Amiens ok
Info for Lille ok
Info for Strasbourg ok
Info for Chateau du Haut Koenigsbourg ok
Info for Colmar ok
Info for Eguisheim ok
Info for Besancon ok
Info for Dijon ok
Info for Annecy ok
Info for Grenoble ok
Info for Lyon ok
Info for Gorges du Verdon ok
Info for Bormes les Mimosas ok
Info for Cassis ok
Info for Marseille ok
Info for Aix en Provence ok
Info for Avignon ok
Info for Uzes ok
Info for Nimes ok
Info for Aigues Mortes ok
Info for Saintes Maries de la mer ok
Info for Collioure ok
Info for Carcassonne ok
Info for Ariege ok
Info for Toulouse ok
Info for Montauban ok
Info for Biarritz ok
Info for Bayonne ok
Info for La Rochelle ok


In [473]:
print("List Lat :", list_lat)
print()
print("Len list lat:", len(list_lat))
print()
print("List Long :", list_lon)
print()
print("Len list lon:", len(list_lon))

List Lat : ['48.6359541', '48.649518', '49.2764624', '49.4938975', '49.4404591', '48.8588897', '49.8941708', '50.6365654', '48.584614', '48.249489800000006', '48.0777517', '48.0447968', '47.2380222', '47.3215806', '45.8992348', '45.1875602', '45.7578137', '43.7496562', '43.1572172', '43.2140359', '43.2961743', '43.5298424', '43.9492493', '44.0121279', '43.8374249', '43.5658225', '43.4522771', '42.52505', '43.2130358', '42.9455368', '43.6044622', '44.0175835', '43.471143749999996', '43.4933379', '46.1591126']

Len list lat: 35

List Long : ['-1.511459954959514', '-2.0260409', '-0.7024738', '0.1079732', '1.0939658', '2.3200410217200766', '2.2956951', '3.0635282', '7.7507127', '7.34429620253195', '7.3579641', '7.3079618', '6.0243622', '5.0414701', '6.1288847', '5.7357819', '4.8320114', '6.3285616', '6.329253867921363', '5.5396318', '5.3699525', '5.4474738', '4.8059012', '4.4196718', '4.3600687', '4.1912837', '4.4287172', '3.0831554', '2.3491069', '1.4065544156065486', '1.4442469', '1.3549

In [474]:
# Create columns in df for coordinates of each cities
df['Latitude'] = None
df['Longitude'] = None

for i in df.index:
    df['Latitude'][i]= float(list_lat[i])
    df['Longitude'][i]= float(list_lon[i])
    
display(df)

,City,Latitude,Longitude
0,Mont Saint Michel,48.636,-1.51146
1,St Malo,48.6495,-2.02604
2,Bayeux,49.2765,-0.702474
3,Le Havre,49.4939,0.107973
4,Rouen,49.4405,1.09397
5,Paris,48.8589,2.32004
6,Amiens,49.8942,2.2957
7,Lille,50.6366,3.06353
8,Strasbourg,48.5846,7.75071
9,Chateau du Haut Koenigsbourg,48.2495,7.3443


In [475]:
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

In [476]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   City       35 non-null     object 
 1   Latitude   35 non-null     float64
 2   Longitude  35 non-null     float64
dtypes: float64(2), object(1)
memory usage: 968.0+ bytes


In [477]:
# extract df in csv file
df.to_csv('src/cities_coordgps.csv',index=False)

### II : Get weather of 35 TOP cities from openweathermap.org API

In [478]:
API_key=""
API_key

'4f87f1b692a3452fb5da3993fa7a9d6d'

In [479]:
paris_coord = [df.Latitude[5],df.Longitude[5]]
paris_coord

[48.8588897, 2.3200410217200766]

In [482]:
## test to get weather for one destination
onecity_test = requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={df.Latitude[5]}&lon={df.Longitude[5]}&exclude=hourly,current,minutely&appid={API_key}&units=metric&lang=fr")
onecity_test = onecity_test.json()
onecity_test

{'lat': 48.8589,
 'lon': 2.32,
 'timezone': 'Europe/Paris',
 'timezone_offset': 3600,
 'daily': [{'dt': 1643544000,
   'sunrise': 1643527428,
   'sunset': 1643561041,
   'moonrise': 1643524260,
   'moonset': 1643552100,
   'moon_phase': 0.93,
   'temp': {'day': 8.29,
    'min': 5.1,
    'max': 9.35,
    'night': 5.71,
    'eve': 6.65,
    'morn': 5.69},
   'feels_like': {'day': 8.29, 'night': 3.8, 'eve': 6.65, 'morn': 4.48},
   'pressure': 1033,
   'humidity': 64,
   'dew_point': 1.64,
   'wind_speed': 3.76,
   'wind_deg': 274,
   'wind_gust': 9.83,
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'ciel dégagé',
     'icon': '01d'}],
   'clouds': 7,
   'pop': 0.06,
   'uvi': 1.15},
  {'dt': 1643630400,
   'sunrise': 1643613750,
   'sunset': 1643647539,
   'moonrise': 1643613780,
   'moonset': 1643643180,
   'moon_phase': 0.97,
   'temp': {'day': 8.2,
    'min': 4,
    'max': 8.28,
    'night': 4,
    'eve': 5.5,
    'morn': 6.48},
   'feels_like': {'day': 4.63, 'nig

In [483]:
# Get weather data for each cities and for keep_keys in the request
df_weather = []
days = 8

for index in range(df.shape[0]):
    for j in range(days):
        r = requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={df.Latitude[index]}&lon={df.Longitude[index]}&exclude=hourly,current,minutely&appid={API_key}&units=metric").json()
        city = df.City[index]
        lat = df.Latitude[index]
        lon = df.Longitude[index]
        day = j
        d = r['daily']
        date = datetime.fromtimestamp(d[j]['dt']).strftime('%d/%m/%Y')
        tem_day = d[j]['temp']['day']
        feelslike_day = d[j]['feels_like']['day']
        pressure = d[j]['pressure']
        humidity = d[j]['humidity']
        wind_speed = d[j]['wind_speed']
        weather_main = d[j]['weather'][0]['main']
        weather_desc = d[j]['weather'][0]['description']
        prob_rain = d[j]['pop']
        clouds = d[j]['clouds']
        uvi = d[j]['uvi']
        df_weather.append([city, lat, lon, day, date, tem_day, feelslike_day, pressure, humidity, wind_speed, weather_main, weather_desc, prob_rain, clouds, uvi])

In [484]:
# Create a dataframe with df_weather
keep_keys = ["city", "Latitude", "Longitude", "day", "date", "tem_day", "feelslike_day", "pressure", "humidity", "wind_speed", "weather_main", "weather_desc", "prob_rain", "clouds", "uvi"]
df_weather_35 = pd.DataFrame(df_weather1, columns=keep_keys)
df_weather_35

,city,Latitude,Longitude,day,date,tem_day,feelslike_day,pressure,humidity,wind_speed,weather_main,weather_desc,prob_rain,clouds,uvi
0,Mont Saint Michel,48.635954,-1.511460,0,30/01/2022,9.34,9.34,1033,73,5.33,Clouds,broken clouds,0.04,61,1.25
1,Mont Saint Michel,48.635954,-1.511460,1,31/01/2022,7.93,4.10,1030,77,8.85,Rain,light rain,0.40,95,0.65
2,Mont Saint Michel,48.635954,-1.511460,2,01/02/2022,9.85,6.87,1028,94,8.08,Rain,light rain,0.20,100,1.46
3,Mont Saint Michel,48.635954,-1.511460,3,02/02/2022,10.78,10.11,1028,84,7.10,Clouds,scattered clouds,0.00,35,1.38
4,Mont Saint Michel,48.635954,-1.511460,4,03/02/2022,7.77,4.90,1018,87,4.76,Clouds,overcast clouds,0.00,100,1.09
5,Mont Saint Michel,48.635954,-1.511460,5,04/02/2022,8.17,4.57,1024,62,7.05,Rain,light rain,0.75,69,2.00
6,Mont Saint Michel,48.635954,-1.511460,6,05/02/2022,10.09,9.11,1030,75,5.54,Rain,light rain,0.36,71,2.00
7,Mont Saint Michel,48.635954,-1.511460,7,06/02/2022,11.23,10.42,1031,77,6.58,Clouds,overcast clouds,0.00,97,2.00
8,St Malo,48.649518,-2.026041,0,30/01/2022,8.54,7.74,1034,76,6.09,Clouds,broken clouds,0.16,57,1.24
9,St Malo,48.649518,-2.026041,1,31/01/2022,8.18,4.12,1031,75,10.30,Clouds,overcast clouds,0.24,99,0.59


In [456]:
df_weather_35.to_csv('src/cities_weather.csv',index=False)

In [457]:
df_weather_35.weather_main.value_counts()

Clouds    121
Rain       79
Clear      63
Snow       17
Name: weather_main, dtype: int64

In [486]:
# Select best cities by weather_main and tem_day
top_cities = df_weather_35[df_weather_35["weather_main"]=="Clear"].sort_values("tem_day", ascending=False).head(5)
list_best_cities = top_cities['city'].unique()

In [488]:
print("According to weather main = Clear and tem_day, we could say that 5 best cities for 8 past days were : ", list_best_cities)

According to weather main = Clear and tem_day, we could say that 5 best cities for 8 past days were :  ['Nimes' 'Aigues Mortes' 'Saintes Maries de la mer' 'Uzes' 'Collioure']


### Get hotels' info about each destination
##### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel

### III. Scraping booking.com

### Store all the information above in a data lake
##### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### IV. Store files on S3 bucket

In [489]:
!pip install Boto3

     |████████████████████████████████| 131 kB 9.7 MB/s eta 0:00:01
  Using cached s3transfer-0.5.0-py3-none-any.whl (79 kB)
     |████████████████████████████████| 8.5 MB 61.6 MB/s eta 0:00:01


In [491]:
import boto3
session = boto3.Session(aws_access_key_id="", 
                        aws_secret_access_key="")

In [495]:
S3 = session.resource('s3')

In [490]:
import csv

In [498]:
# put objets on the creating bucket 
object = S3.Object('fesson-bucket-kayak', 'cities_coordgps.csv').put(Body=open("src/cities_coordgps.csv", "rb"))

In [499]:
object2 = S3.Object('fesson-bucket-kayak','cities_weather.csv').put(Body=open("src/cities_weather.csv", "rb"))

### Extract, transform and load cleaned data from your datalake to a data warehouse
##### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

#### ID Connection to the RDS DATABASE MYSQL :

    Host : fesson-kayak-db.cfemolqokjpg.eu-west-3.rds.amazonaws.com
    Port : 3306
    User : fessonuser
    Password :     db : kayak